========================================
   
__Contents__
* Search usage
   0. Import module & Load data
   1. Defining parameter search space
   2. Defining feature search space (optional)
   3. Run search
   
* Log usage
   1. Extract pramater & feature setting
   2. Make meta feature for stacking
   
========================================

# Search usage

## 0. Import module & Load data
In here, use the breast cancer wisconsin dataset to modeling.   
This is binary classification dataset.   
Firstly, this dataset is splitted to three datasets(Train, Test)

In [ ]:
import os 
import numpy as np, pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression


from bokeh.io import output_notebook
output_notebook() # When you need search visualization, need run output_notebook()

from hyperopt import hp, tpe, rand
from cvopt.model_selection import hyperoptCV

dataset = datasets.load_breast_cancer()
Xtrain, Xtest, ytrain, ytest = train_test_split(dataset.data, dataset.target, test_size=0.3, random_state=0)

print("Train features shape:", Xtrain.shape)
print("Test features shape:", Xtest.shape)

## 1. Defining parameter search space
In cvopt, a Paraneter search space definition depend on base module of cross validation class.   
When use hyperoptCV, you define [hyperopt format search space](https://github.com/hyperopt/hyperopt/wiki/FMin#2-defining-a-search-space).

In [ ]:
param_distributions = {
    "penalty": hp.choice("penalty", ['l1', 'l2']),
    "C": hp.loguniform("C", -3, 3), 
    "tol" : hp.loguniform("tol", -4, -2), 
    "class_weight" : hp.choice("class_weight", [None, "balanced"]),
    }

## 2. Defining feature search space (optional)
Features are selected per feature group.   
__If feature group is set "-100", this group's features always are used.__   
Criterion of separating group is, for example, random, difference of feature engineering method or difference of data source.
   
When you don't set featured group, optimizer use all input features.

------------------------------------

### Example.   
When data has 5 features(5 cols) and feature group is set as shown below.
   
| feature index(data col index) | feature group |   
|:------------:|:------------:|   
| 0 | 0 |
| 1 | 0 |
| 2 | 0 |
| 3 | 1 |
| 4 | 1 |
   
Define as follows python's list.   
   
```feature_groups = [0, 0, 0, 1, 1]```

As search result, you may get flag per feature group.
   
```feature_groups0: True   
feature_groups1: False```
   
This result means that optimizer recommend using group 1 features(col index:0,1,2) and not using group 2.

------------------------------------


In [ ]:
feature_groups = np.random.randint(0, 5, Xtrain.shape[1]) 

## 3. Run search
cvopt has API like scikit-learn cross validation class.   
When you had use scikit-learn, you can use cvopt very easy.
   
For each optimizer class's detail, please see [API reference]().

In [ ]:
estimator = LogisticRegression()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
hpcv = hyperoptCV(estimator, param_distributions, 
                  scoring="roc_auc",               # Objective of search
                  cv=cv,                           # Cross validation setting
                  max_evals=32,                    # Number of search
                  n_jobs=3,                        # Number of jobs to run in parallel.
                  random_state=0,                  # seed in hyperopt
                  verbose=2,                       # 0: don't display status, 1:display status by stdout, 2:display status by graph 
                  logdir="./search_usage",         # If this path is specified, save the log.
                  model_id="search_usage",         # used estimator's dir and file name in save.
                  save_estimator=2,
                  )

hpcv.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), 
         # validation_data is optional.
         # This data is only used to compute validation score(don't fit).
         # When this data is input & save_estimator=True,the estimator which is fitted whole Xtrain is saved.
         feature_groups=feature_groups)
ytest_pred = hpcv.predict(Xtest)

In [ ]:
pd.DataFrame(hpcv.cv_results_).head() # Search results

# Log usage

## 1. Extract pramater & feature setting
In cvopt, helper function is Included to handle log file easily.   
When you want to extract settings from log file, It can be implemented as follows.

In [ ]:
from cvopt.utils import extract_params
estimator_params, feature_params, feature_select_flag  = extract_params(logdir="./search_usage", 
                                                                        model_id="search_usage", 
                                                                        target_index=0, 
                                                                        feature_groups=feature_groups)

estimator.set_params(**estimator_params)         # Set estimator parameters
Xtrain_selected = Xtrain[:, feature_select_flag] # Extract selected feature columns

print(estimator)
print("Train features shape:", Xtrain.shape)
print("Train selected features shape:",Xtrain_selected.shape)

## 2. Make meta feature for stacking
When you want to male mete feature and [stacking](https://mlwave.com/kaggle-ensembling-guide/), It can be implemented as follows.   
   
When run search, You need set "save_estimator>0" to make meta feature.    
In addition, you need set "save_estimator>1" to make meta feature from the data which is not fitted.

In [ ]:
from cvopt.utils import mk_metafeature
Xtrain_meta, Xtest_meta = mk_metafeature(Xtrain, ytrain, 
                                         logdir="./search_usage", 
                                         model_id="search_usage", 
                                         target_index=0, 
                                         cv=cv, 
                                         validation_data=(Xtest, ytest), 
                                         feature_groups=feature_groups, 
                                         estimator_method="predict_proba")

print("Train features shape:", Xtrain.shape)
print("Train meta features shape:", Xtrain_meta.shape)
print("Test features shape:", Xtest.shape)
print("Test meta features shape:",  Xtest_meta.shape)